<a href="https://colab.research.google.com/github/manukala6/processing/blob/master/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [0]:
import os
import ee
import numpy as np
import requests
import zipfile
from sklearn.linear_model import LinearRegression
from osgeo import gdal
from osgeo import osr

**Authenticate and Initialize**

In [0]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/tgGCtaKpU__Xchv4qatHMZD_ksl0Ix6ni_WF0DsuuswnjZ2lOHHjQCg

Successfully saved authorization token.


In [0]:
ee.Initialize()

**Collect Imagery from Assets**

In [0]:
# links to earth engine image assets
naiman_image_links = [
  'users/manukala6/naimanResiduals/naimanResiduals1985',
  'users/manukala6/naimanResiduals/naimanResiduals1986',
  'users/manukala6/naimanResiduals/naimanResiduals1987',
  'users/manukala6/naimanResiduals/naimanResiduals1988',
  'users/manukala6/naimanResiduals/naimanResiduals1989',
  'users/manukala6/naimanResiduals/naimanResiduals1991',
  'users/manukala6/naimanResiduals/naimanResiduals1992',
  'users/manukala6/naimanResiduals/naimanResiduals1993',
  'users/manukala6/naimanResiduals/naimanResiduals1994',
  'users/manukala6/naimanResiduals/naimanResiduals1995',
  'users/manukala6/naimanResiduals/naimanResiduals1996',
  'users/manukala6/naimanResiduals/naimanResiduals1997',
  'users/manukala6/naimanResiduals/naimanResiduals1998',
  'users/manukala6/naimanResiduals/naimanResiduals1999',
  'users/manukala6/naimanResiduals/naimanResiduals2000',
  'users/manukala6/naimanResiduals/naimanResiduals2001',
  'users/manukala6/naimanResiduals/naimanResiduals2002',
  'users/manukala6/naimanResiduals/naimanResiduals2003',
  'users/manukala6/naimanResiduals/naimanResiduals2004',
  'users/manukala6/naimanResiduals/naimanResiduals2005']

# earth engine geometry of study area
study_area = ee.Geometry.Polygon(
        [[[120.58601753616483, 43.18954310017519],
          [120.58601753616483, 43.091842417767054],
          [120.7322730293289, 43.091842417767054],
          [120.7322730293289, 43.18954310017519]]])

In [0]:
naiman_images = []
for image_link in naiman_image_links:
  naiman_images.append(ee.Image(image_link))

In [0]:
img_test = naiman_images[0]
url = img_test.getDownloadUrl({'name': 'Test1', 'crs': 'EPSG:4326', 'scale': '30'})
url

'https://earthengine.googleapis.com/api/download?docid=22e4096d449ae78b9372afa790e6f466&token=4da6015ea83c37e51b0b6f5bf9ad0f8d'

In [0]:
r = requests.get(url, stream=True)
with open('filename', 'wb') as fd:
    for chunk in r.iter_content(chunk_size=1024):
        fd.write(chunk)
z = zipfile.ZipFile('filename')
z.extractall()

**Create an array of shape** `(num_years, num_bands, num_rows, num_cols)`

---



In [0]:
# function to generate numpy array images from image collection
def convert_ee2arr (ee_image):
  # gather decimal coordinates per pixel
  lat_lon = ee_image.clip(study_area).pixelLonLat().addBands(ee_image)
  lat_lon = lat_lon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=study_area,
      scale=30,
      maxPixels=10000000000
  )
  # convert each band to nd array
  ndvi = np.array((ee.Array(lat_lon.get('ndviCalendar')).getInfo()))
  precip = np.array((ee.Array(lat_lon.get('precipitation')).getInfo()))
  latitude = np.array((ee.Array(lat_lon.get('latitude')).getInfo()))
  longitude = np.array((ee.Array(lat_lon.get('longitude')).getInfo()))
  # reshape to rows/columns
  num_rows = np.unique(latitude, return_counts=True)[1][0].astype(int)
  num_cols = np.unique(longitude, return_counts=True)[1][0].astype(int)
  ndvi = ndvi.reshape(num_rows, num_cols)
  precip = precip.reshape(num_rows, num_cols)
  # stack band arrays
  return np.stack((ndvi, precip))

In [0]:
# do the thing
naiman_arr_images = []
for i in range(0,15):
  naiman_arr_images.append(convert_ee2arr(naiman_images[i]))

In [0]:
# stack into one array of shape len(naiman_images)
naiman_arr = np.asarray(naiman_arr_images)

In [0]:
# check the shape
print('number of years: ', naiman_arr.shape[0])
print('number of bands: ', naiman_arr.shape[1])
print('number of cols:  ', naiman_arr.shape[2])
print('number of rows:  ', naiman_arr.shape[3])

number of years:  15
number of bands:  2
number of cols:   543
number of rows:   362


**Regression**

In [0]:
# obtain ndvi band only
ndvi = naiman_arr[:,0]

# verify shape
ndvi.shape

(15, 543, 362)

In [0]:
# append slope per pixel into nd array
slopes = np.array([])
for row in range(300):
  for col in range(300):
    x = np.arange(15).reshape((-1,1))
    y = ndvi[:,col,row]
    model = LinearRegression().fit(x, y)
    slopes = np.append(slopes,model.coef_[0])

In [0]:
# verify shape
slopes = slopes.reshape((300,300))
slopes.shape

(300, 300)

In [0]:
x = np.asarray([0,1,2,3,4]).reshape((-1,1))
y = ndvi[:,0,0]

model = LinearRegression().fit(x, y)

# slope for this pixel
slope = model.coef_[0]
print(slope)

0.015169249475002297


**Save as GeoTIFF**

---



In [0]:
# config
INPUT_ARR = naiman_arr[0,0]
upper_left_tuple = (120.58601753616483, 43.18954310017519)
# create gdal data source raster object
gdal_driver = gdal.GetDriverByName('GTiff')
output_raster = gdal_driver.Create('sample_raster_12.tif', int(INPUT_ARR.shape[1]), int(INPUT_ARR.shape[0]), 1, gdal.GDT_Float64)
geotransform = (upper_left_tuple[0], 30, upper_left_tuple[1] + 30, -1 *(30), 0, 0)
spatial_reference = osr.SpatialReference()
spatial_reference.ImportFromEPSG(4326)
output_raster.SetProjection(spatial_reference.ExportToWkt())
output_raster.SetGeoTransform(geotransform)
output_band = output_raster.GetRasterBand(1)
output_band.SetNoDataValue(15)
output_band.WriteArray(INPUT_ARR)          
output_band.FlushCache()
output_band.ComputeStatistics(False)
output_raster

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7fcc666c8bd0> >

In [0]:
output_raster.GetLayerCount()

0